In [ ]:
# !pip install ipywidgets

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

### Import the MNIST dataset

In [ ]:
(X_train, y_train), (X_test, y_test) =  tf.keras.datasets.mnist.load_data()

print("Train data shape:", X_train.shape)
print("Train labels shape:", y_train.shape)

print("Test data shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

### Examine the dataset

In [ ]:
labels = range(10)
n_classes = len(labels)
print("Number of classes:", n_classes)
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(18,8))

n_columns = 12
n_rows = 4

for i in range(1,n_columns*n_rows+1):
    fig.add_subplot(n_rows, n_columns, i)
    plt.imshow(X_train[i], cmap="gray")
    plt.title(labels[y_train[i]])
    # Turn off tick labels
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
W = H = 28
BATCH_SIZE = 128


def normalizer(image, label):
    image = 2 * (tf.to_float(image)) / 255 - 1.
    return image, label


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(normalizer).shuffle(1000).batch(BATCH_SIZE)

train_dataset

### The Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, ReLU, PReLU, Activation, Flatten,
    Dropout, UpSampling2D, Convolution2D, LeakyReLU,
    Input, BatchNormalization, Reshape,
)
from tensorflow.keras.models import Sequential

Z_SIZE = 64
ALPHA = 0.2
BETA1 = 0.5
SMOOTH = 0.1

def get_generator_model():
    n_features = 128
    return Sequential([
        Dense(n_features, input_shape=[Z_SIZE]),
        LeakyReLU(ALPHA),
        Dense(n_features * 2),
        LeakyReLU(ALPHA),
        Dense(n_features * 4),
        LeakyReLU(ALPHA),
        Dense(W * H),
        Activation('tanh'),
        Reshape((W, H))
    ])

def get_discriminator_model():
    n_features = 128
    return Sequential([
        Flatten(input_shape=(W, H)),
        Dense(n_features * 4),
        LeakyReLU(ALPHA),
        Dense(n_features * 2),
        LeakyReLU(ALPHA),
        Dense(n_features),
        LeakyReLU(ALPHA),
        Dense(1),
        Activation('sigmoid')
    ])

print("Generator: "); get_generator_model().summary()
print("\nDiscriminator: "); get_discriminator_model().summary()

### Training & Evaluation

In [ ]:
import numpy as np

def denormalizer(img): 
    return ((img + 1)*255 / 2).astype(np.uint8)

def display_images(dataset, figsize=(6,6)):
    fig, axes = plt.subplots(figsize[0], figsize[1], sharex=True, sharey=True, figsize=figsize,)
    for ii, ax in enumerate(axes.flatten()):
        img = dataset[ii,:,:]
        img = denormalizer(img) # Scale back to 0-255
        ax.imshow(img, aspect='equal', cmap="gray")
      
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()


from tensorflow.keras.backend import binary_crossentropy
def discriminator_loss(outputs_real, outputs_fake):
  # for the real image from the training set, we want them to be classified as positives,  
  # so we want their labels to be all ones. 
    loss_real = tf.reduce_mean(
        binary_crossentropy(
            output=outputs_real, 
            target=tf.ones_like(outputs_real)
       )
    )

  # for the fake images produced by the generator, we want the discriminator to classify them as false images,
  # so we set their labels to be all zeros.
    loss_fake = tf.reduce_mean(
        tf.keras.backend.binary_crossentropy(
            output=outputs_fake, 
            target=tf.zeros_like(outputs_fake)
        )
    )
    d_loss = tf.stack(loss_real + loss_fake, 0)
    return d_loss
    
def generator_loss(d_logits_fake):
    # since the generator wants the discriminator to output 1s for its images, it uses the discriminator logits for the
    # fake images and assign labels of 1s to them.
    g_loss = tf.reduce_mean(
        tf.keras.backend.binary_crossentropy(
            output=d_logits_fake, 
            target=tf.ones_like(d_logits_fake)
        )
    ) 
    return g_loss

from tensorflow.train import AdamOptimizer
import time
def train(epochs, gen_net, dis_net):
    generator_optimizer = AdamOptimizer(LEARNING_RATE, BETA1)
    discriminator_optimizer = AdamOptimizer(LEARNING_RATE, BETA1)

    # generate sample noise for evaluation
    fake_input_test = tf.random_uniform(shape=(6 * 6, Z_SIZE),
                                     minval=-1.0, maxval=1.0, dtype=tf.float32)
    
    for epoch in range(1, epochs + 1):
        start = time.time()

        for batch_real_images, batch_real_labels in train_dataset:
            curr_batch_size = tf.shape(batch_real_labels)[0]

            fake_input = tf.random_uniform(
                shape=(curr_batch_size, Z_SIZE), minval=-1.0, maxval=1.0, dtype=tf.float32
            )
            with tf.GradientTape(persistent=True) as tape:

                # run the generator with the random noise batch
                g_images = gen_net(fake_input, training=True)

                # run the discriminator with real and generated images as inputs 
                d_logits = dis_net(tf.concat([batch_real_images, g_images], axis=0), training=True)

                d_logits_real = d_logits[:curr_batch_size]
                d_logits_fake = d_logits[curr_batch_size:]

                # compute the generator loss
                gen_loss = generator_loss(d_logits_fake)

                # compute the discriminator loss
                dis_loss = discriminator_loss(d_logits_real, d_logits_fake)

            discriminator_grads = tape.gradient(dis_loss, dis_net.variables)
            generator_grads = tape.gradient(gen_loss, gen_net.variables)

            discriminator_optimizer.apply_gradients(zip(discriminator_grads, dis_net.variables))
            generator_optimizer.apply_gradients(zip(generator_grads, gen_net.variables))

        
        generated_samples = gen_net(fake_input_test, training=False)
#         from IPython.display import clear_output
#         clear_output()
        display_images(generated_samples.numpy())
        print('epoch: ',  epoch,
              ', g_loss: ', gen_loss.numpy(), 
              ', d_loss: ', dis_loss.numpy(), 
              ', time: ', time.time() - start
             )

In [ ]:
LEARNING_RATE = 0.0002
EPOCHS = 35

generator_net = get_generator_model()
discriminator_net = get_discriminator_model()

train(EPOCHS, generator_net, discriminator_net)

### DCGAN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, ReLU, PReLU, Activation, Flatten, Average,
    Dropout, UpSampling2D, Conv2D, LeakyReLU, Conv2DTranspose,
    Input, BatchNormalization, Reshape, UpSampling2D, GlobalAveragePooling2D
)
from tensorflow.keras.models import Sequential

Z_SIZE = 64
ALPHA = 0.2
BETA1 = 0.5

def get_dcgan_generator_model():
    return Sequential([
        Dense((W / 4) * (H / 4) * 64, input_shape=[Z_SIZE], use_bias=False),
        BatchNormalization(),
        LeakyReLU(ALPHA),
        Reshape(((W / 4), (H / 4), 64)),
        Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(ALPHA),
        Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(ALPHA),
        Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        Activation('tanh'),
        Reshape((W, H))
    ])

def get_dcgan_discriminator_model():
    return Sequential([
        Reshape((W, H, 1), input_shape=(W, H)),
        Convolution2D(64, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(ALPHA),
        Dropout(0.3),
        Convolution2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(ALPHA),
        Dropout(0.3),
        Flatten(),
        Dense(1),
        Activation('sigmoid')
    ])

dcgan_generator_net = get_dcgan_generator_model()
dcgan_discriminator_net = get_dcgan_discriminator_model()

print("DCGAN Generator: "); dcgan_generator_net.summary()
print("\nDCGAN Discriminator: "); dcgan_discriminator_net.summary()

In [ ]:
LEARNING_RATE = 0.0002
EPOCHS = 35

dcgan_generator_net = get_dcgan_generator_model()
dcgan_discriminator_net = get_dcgan_discriminator_model()

train(EPOCHS, dcgan_generator_net, dcgan_discriminator_net)

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

ref = np.random.random((Z_SIZE, 1, Z_SIZE)) * 2 - 1

def f(z_prior):
    plt.figure(figsize=(1,1))
    fake_input_test = ref[z_prior, :]
    
    img = dcgan_generator_net(fake_input_test, training=False)[0].numpy()
    img = denormalizer(img)
    plt.axis('off')
    plt.imshow(img, aspect='equal', cmap="gray")

interactive_plot = interactive(f, z_prior=(0, Z_SIZE - 1))
output = interactive_plot.children[-1]
interactive_plot
# f(9)

### Conditional DCGAN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, ReLU, PReLU, Activation, Flatten, Embedding, multiply,
    Dropout, UpSampling2D, Conv2D, LeakyReLU, Conv2DTranspose,
    Input, BatchNormalization, Reshape, UpSampling2D, GlobalAveragePooling2D
)
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model

Z_SIZE = 64
ALPHA = 0.2
BETA1 = 0.5

class CGenerator(Model):
    def __init__(self):
        super(CGenerator, self).__init__(name='')
        self.label = Sequential([
            Embedding(10, Z_SIZE, input_shape=[1]),
            Flatten()
        ])
        self.z_inp = Flatten(input_shape=[Z_SIZE])
        self.rest = Sequential([
            Dense((W / 4) * (H / 4) * 64, use_bias=False),
            BatchNormalization(),
            LeakyReLU(ALPHA),
            Reshape(((W / 4), (H / 4), 64)),
            Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False),
            BatchNormalization(),
            LeakyReLU(ALPHA),
            Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
            BatchNormalization(),
            LeakyReLU(ALPHA),
            Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False),
            Activation('tanh'),
            Reshape((W, H))
        ])
    def call(self, lbl, z, training):
        x = multiply([self.z_inp(z), self.label(lbl)])
        x = self.rest(x, training=training)
        return x
    
class CDiscriminator(Model):
    def __init__(self):
        super(CDiscriminator, self).__init__(name='')
        self.label = Sequential([
            Embedding(10, H * W, input_shape=[1]),
            Flatten()
        ])
        self.img_inp = Flatten(input_shape=[W, H])
        self.rest = Sequential([
            Reshape((W, H, 1), input_shape=(W, H)),
            Convolution2D(64, (5, 5), strides=(2, 2), padding='same'),
            LeakyReLU(ALPHA),
            Dropout(0.3),
            Convolution2D(128, (5, 5), strides=(2, 2), padding='same'),
            LeakyReLU(ALPHA),
            Dropout(0.3),
            Flatten(),
            Dense(1),
            Activation('sigmoid')
        ])
    def call(self, lbl, img, training):
        x = multiply([self.img_inp(img), self.label(lbl)])
        x = self.rest(x, training=training)
        return x

cdcgan_generator_net = CGenerator()
cdcgan_generator_net(tf.ones([1, 1]), tf.random_uniform(shape=[1, Z_SIZE]), training=False)
cdcgan_discriminator_net = CDiscriminator()
cdcgan_discriminator_net(tf.ones([1, 1]), tf.random_uniform(shape=[1, W, H]), training=False)

print("C-DCGAN Generator: "); cdcgan_generator_net.layers[-1].summary()
print("\nC-DCGAN Discriminator: "); cdcgan_discriminator_net.layers[-1].summary()

In [ ]:

from tensorflow.train import AdamOptimizer
import time
def train_conditional(epochs, gen_net, dis_net):
    generator_optimizer = AdamOptimizer(LEARNING_RATE, BETA1)
    discriminator_optimizer = AdamOptimizer(LEARNING_RATE, BETA1)

    # generate sample noise for evaluation
    fake_input_test = tf.random_uniform(shape=(6 * 6, Z_SIZE), minval=-1.0, maxval=1.0, dtype=tf.float32)
    fake_input_test_labels = tf.cast(tf.random_uniform(shape=[6 * 6, 1]) * 10, tf.uint8)
    
    for epoch in range(1, epochs + 1):
        start = time.time()

#         from tqdm import tqdm
        for batch_real_images, batch_real_labels in train_dataset:            
            curr_batch_size = tf.shape(batch_real_labels)[0]
            batch_real_labels = tf.reshape(batch_real_labels, [curr_batch_size, 1])
            fake_input = tf.random_uniform(
                shape=(curr_batch_size, Z_SIZE), minval=-1.0, maxval=1.0, dtype=tf.float32
            )
            fake_input_labels = tf.cast(tf.random_uniform(shape=[curr_batch_size, 1]) * 10, tf.uint8)
            
            with tf.GradientTape(persistent=True) as tape:

                # run the generator with the random noise batch
                g_images = gen_net(fake_input_labels, fake_input, training=True)

                # run the discriminator with real and generated images as inputs 
                d_logits = dis_net(
                    tf.concat([batch_real_labels, fake_input_labels], axis=0), 
                    tf.concat([batch_real_images, g_images], axis=0), training=True)

                d_logits_real = d_logits[:curr_batch_size]
                d_logits_fake = d_logits[curr_batch_size:]

                # compute the generator loss
                gen_loss = generator_loss(d_logits_fake)

                # compute the discriminator loss
                dis_loss = discriminator_loss(d_logits_real, d_logits_fake)

            discriminator_grads = tape.gradient(dis_loss, dis_net.variables)
            generator_grads = tape.gradient(gen_loss, gen_net.variables)

            discriminator_optimizer.apply_gradients(zip(discriminator_grads, dis_net.variables))
            generator_optimizer.apply_gradients(zip(generator_grads, gen_net.variables))

        
        generated_samples = gen_net(fake_input_test_labels, fake_input_test, training=False)
#         from IPython.display import clear_output
#         clear_output()
        display_images(generated_samples.numpy())
        print('epoch: ',  epoch,
              ', g_loss: ', gen_loss.numpy(), 
              ', d_loss: ', dis_loss.numpy(), 
              ', time: ', time.time() - start
             )
        
LEARNING_RATE = 0.0002
EPOCHS = 35

cdcgan_generator_net = CGenerator()
cdcgan_discriminator_net = CDiscriminator()

train_conditional(EPOCHS, cdcgan_generator_net, cdcgan_discriminator_net)

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

ref = tf.random_uniform((Z_SIZE, 1, Z_SIZE)) * 2 - 1

def f(digit, z_prior):
    plt.figure(figsize=(1,1))
    fake_input_test = ref[z_prior, :]
    
    img = cdcgan_generator_net(tf.constant([digit]), fake_input_test, training=False)[0].numpy()
    img = denormalizer(img)
    plt.axis('off')
    plt.imshow(img, aspect='equal', cmap="gray")

# interactive_plot = interactive(f, digit=(0, 9), z_prior=(0, Z_SIZE - 1))
# output = interactive_plot.children[-1]
# interactive_plot
f(8, 12)